# Evaluation Review

This notebook may be used to quickly view the results of an evaluation run.

In [1]:
import os
from ast import literal_eval

import pandas as pd

## Load Evaluation CSV

In [2]:
EVALS_PATH = "evals/output/"

In [3]:
eval_csv = [x for x in os.listdir(EVALS_PATH) if x.endswith(".csv") and "benchmark" in x]
eval_csv.sort()

In [4]:
eval_csv

['eval_benchmark_results_2025-09-16_14-32-03.csv']

In [5]:
CSV_NAME = eval_csv[-1]

In [6]:
data = pd.read_csv(
    EVALS_PATH + CSV_NAME,
    converters={
        "generated_cypher": literal_eval,
        "called_tools": literal_eval,
    },
)

In [7]:
print(f"Evaluation Questions: {len(data)}")

Evaluation Questions: 10


In [12]:
data.head(5)

,question_id,question,expected_answer,agent_final_answer,generated_cypher,model,available_tools,called_tools,num_messages,num_llm_calls,num_tool_calls,response_time,error
0,top_rated_movies,What are the top 5 highest-rated movies by IMD...,"1. Band of Brothers (9.6), 2. Civil War, The (...",The top 5 highest-rated movies by IMDb rating ...,[{'query': 'MATCH (m:Movie) WHERE m.imdbRating...,openai:gpt-4.1,"['get_neo4j_schema', 'read_neo4j_cypher', 'fin...","[{'name': 'get_neo4j_schema', 'args': {}, 'id'...",6,3,2,5.341379,NaN
1,most_prolific_actors,Which actors have appeared in the most movies?,"1. Robert De Niro (56 movies), 2. Bruce Willis...",The actors who have appeared in the most movie...,[{'query': 'MATCH (a:Actor)-[:ACTED_IN]->(m:Mo...,openai:gpt-4.1,"['get_neo4j_schema', 'read_neo4j_cypher', 'fin...","[{'name': 'get_neo4j_schema', 'args': {}, 'id'...",6,3,2,6.188545,NaN
2,actor_director_same_movie,What movies did Tom Hanks direct and act in?,Larry Crowne (2011) - Tom Hanks both directed ...,Tom Hanks both directed and acted in the movie...,[{'query': 'MATCH (p:Person {name: 'Tom Hanks'...,openai:gpt-4.1,"['get_neo4j_schema', 'read_neo4j_cypher', 'fin...","[{'name': 'get_neo4j_schema', 'args': {}, 'id'...",6,3,2,5.602856,NaN
3,best_genre_by_rating,Which genre has the highest average IMDb rating?,Documentary has the highest average IMDb ratin...,The genre with the highest average IMDb rating...,[{'query': 'MATCH (g:Genre)<-[:IN_GENRE]-(m:Mo...,openai:gpt-4.1,"['get_neo4j_schema', 'read_neo4j_cypher', 'fin...","[{'name': 'get_neo4j_schema', 'args': {}, 'id'...",8,4,3,5.424072,NaN
4,expensive_90s_movies,What are the most expensive movies (by budget)...,"1. Titanic (1997) - $200M, 2. Waterworld (1995...",The most expensive movies (by budget) from the...,[{'query': 'MATCH (m:Movie) WHERE m.year >= 19...,openai:gpt-4.1,"['get_neo4j_schema', 'read_neo4j_cypher', 'fin...","[{'name': 'get_neo4j_schema', 'args': {}, 'id'...",6,3,2,6.805763,NaN


In [9]:
data.iloc[0, 7][1]['args'].keys()

dict_keys(['query'])

## Report

In [10]:
from scripts.generate_report import pretty_print

In [11]:
for i in range(len(data)):
    pretty_print(data.iloc[i])

Question: What are the top 5 highest-rated movies by IMDb rating?
Model: openai:gpt-4.1
Available Tools: ['get_neo4j_schema', 'read_neo4j_cypher', 'find_movie_recommendations']
Number of Messages: 6
Number of LLM Calls: 3
Number of Tool Calls: 2
Called Tools: get_neo4j_schema-{}, read_neo4j_cypher-{'query': 'MATCH (m:Movie)\nWHERE m.imdbRating IS NOT NULL\nRETURN m.title AS title, m.imdbRating AS imdbRating, m.imdbVotes AS imdbVotes, m.year AS year\nORDER BY m.imdbRating DESC, m.imdbVotes DESC\nLIMIT 5'}
Text2Cypher Queries:
1.)
MATCH (m:Movie)
WHERE m.imdbRating IS NOT NULL
RETURN m.title AS title, m.imdbRating AS imdbRating, m.imdbVotes AS imdbVotes, m.year AS year
ORDER BY m.imdbRating DESC, m.imdbVotes DESC
LIMIT 5
Final Answer: The top 5 highest-rated movies by IMDb rating in the database are:

1. Band of Brothers (2001)
   - IMDb Rating: 9.6 (213,089 votes)

2. The Civil War (1990)
   - IMDb Rating: 9.5 (4,942 votes)

3. The Shawshank Redemption (1994)
   - IMDb Rating: 9.3 (1,62